In [61]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [62]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_column', None)
df_engagement = pd.read_csv('../data/engagement_clusters.csv', na_values=['?', None, 'undefined'])
df_engagement.sample(5)

,MSISDN/Number,Bearer Id,Dur. (ms),Total Data,cluster
35159,33659329654,1,107635.0,7.906786e+08,1
67269,33667931526,2,193280.0,9.841935e+08,0
32734,33658821425,2,286230.0,6.229598e+08,0
19440,33641064265,3,276359.0,1.180369e+09,0
64433,33667012279,1,86399.0,8.056324e+08,1


In [63]:
df_experience = pd.read_csv('../data/experience_clusters.csv', na_values=['?', None, 'undefined'])
df_experience.sample(5)

,MSISDN/Number,Total RTT,Total TCP retransmission,Total Throughput,Handset Type,encoded,cluster
29229,33698245199,51.0,3051202.0,53632.0,Samsung Galaxy S5 (Sm-G900F),713,0
7037,33650360830,92.0,11606.0,117841.0,Huawei P10,313,1
30124,33698976581,95.0,3029800.0,19426.0,Huawei B528S-23A,232,1
28012,33684278761,69.0,1091408.0,70770.0,Apple iPhone 7 Plus (A1784),54,1
26982,33676702200,55.0,2175387.0,43019.0,Samsung Galaxy S8 (Sm-G950F),725,0


In [64]:
# how many missing values exist or better still what is the % of missing values in the dataset?
def percent_missing(df):

    # Calculate total number of cells in dataframe
    totalCells = np.product(df.shape)

    # Count number of missing values per column
    missingCount = df.isnull().sum()

    # Calculate total number of missing values
    totalMissing = missingCount.sum()

    # Calculate percentage of missing values
    print("The dataset contains", round(((totalMissing/totalCells) * 100), 2), "%", "missing values.")

percent_missing(df_engagement)
percent_missing(df_experience)

The dataset contains 0.0 % missing values.
The dataset contains 0.0 % missing values.


In [65]:
less_engaged_centroid = df_engagement[df_engagement['cluster'] == 0].groupby('cluster').mean()
less_engaged_centroid

,MSISDN/Number,Bearer Id,Dur. (ms),Total Data
cluster,,,,
0,3.367435e+10,2.182889,215418.810195,1.116296e+09


In [66]:
def compute_engagement_score(df,centroid):
    x = float(centroid['Bearer Id'])
    y = float(centroid['Dur. (ms)'])
    z = float(centroid['Total Data'])
    df['engagement score'] = ((df['Bearer Id'] - x)**2 + (df['Dur. (ms)'] - y)**2 + (df['Total Data'] - z)**2)**0.5
    return df
df_engagement = compute_engagement_score(df_engagement,less_engaged_centroid)

In [67]:
df_engagement.head()

,MSISDN/Number,Bearer Id,Dur. (ms),Total Data,cluster,engagement score
0,3197020876596,1,877385.0,232123971.0,1,8.841722e+08
1,33601001722,1,116720.0,878690574.0,1,2.376053e+08
2,33601001754,1,181230.0,156859643.0,1,9.594363e+08
3,33601002511,1,134969.0,595966483.0,1,5.203294e+08
4,33601007832,1,49878.0,422320698.0,1,6.939752e+08


In [68]:
worst_experience_centroid = df_experience[df_experience['cluster'] == 0]\
    [['Total RTT', 'Total TCP retransmission', 'Total Throughput', 'encoded', 'cluster']].groupby('cluster').mean()
worst_experience_centroid.head()

,Total RTT,Total TCP retransmission,Total Throughput,encoded
cluster,,,,
0,245.546023,4.195681e+06,32562.62303,706.700761


In [69]:
def compute_experience_score(df,centroid):
    x = float(centroid['Total RTT'])
    y = float(centroid['Total TCP retransmission'])
    z = float(centroid['Total Throughput'])
    w = float(centroid['encoded'])
    df['experience score'] = ((df['Total RTT'] - x)**2 + (df['Total TCP retransmission'] - y)**2 \
                              + (df['Total Throughput'] - z)**2 + (df['encoded'] - w)**2)**0.5
    return df
df_experience = compute_experience_score(df_experience,worst_experience_centroid)
df_experience.head()

,MSISDN/Number,Total RTT,Total TCP retransmission,Total Throughput,Handset Type,encoded,cluster,experience score
0,33601008617,91.0,9370832.0,56781.0,Apple iPhone Se (A1723),62,1,5.175208e+06
1,33601011634,39.0,110232.0,42416.0,Huawei Mate 10 Pro Porsche Design Huawei Mate 10,301,1,4.085460e+06
2,33601021217,160.0,14594645.0,19256.0,Apple iPhone 7 Plus (A1784),54,1,1.039897e+07
3,33601031129,60.0,2325497.0,38190.0,Apple iPhone 8 Plus (A1897),60,1,1.870192e+06
4,33601034530,656.0,2006261.0,8539.0,Apple iPhone 7 (A1778),51,1,2.189551e+06


In [70]:
df_satisfaction = pd.merge(df_engagement[['MSISDN/Number', 'engagement score']], df_experience[['MSISDN/Number', 'experience score']], on="MSISDN/Number")
df_satisfaction.head()

,MSISDN/Number,engagement score,experience score
0,33601008617,3.411151e+08,5.175208e+06
1,33601011634,4.615728e+08,4.085460e+06
2,33601021217,4.862035e+08,1.039897e+07
3,33601031129,4.792428e+08,1.870192e+06
4,33601034530,5.164648e+08,2.189551e+06


In [71]:
df_satisfaction['Satisfaction Score'] = (df_satisfaction['engagement score'] + df_satisfaction['experience score'])/2

In [72]:
df_satisfaction.head()

,MSISDN/Number,engagement score,experience score,Satisfaction Score
0,33601008617,3.411151e+08,5.175208e+06,1.731452e+08
1,33601011634,4.615728e+08,4.085460e+06,2.328292e+08
2,33601021217,4.862035e+08,1.039897e+07,2.483012e+08
3,33601031129,4.792428e+08,1.870192e+06,2.405565e+08
4,33601034530,5.164648e+08,2.189551e+06,2.593272e+08


### Top 10 satisfied customers

In [73]:
df_satisfaction[['MSISDN/Number', 'Satisfaction Score']].sort_values(by="Satisfaction Score", ascending=False).head(10)

,MSISDN/Number,Satisfaction Score
23613,33667725464,4.719602e+09
10863,33659084281,4.237100e+09
34316,33762644658,4.011219e+09
1720,33614892860,3.865531e+09
32250,33760536639,3.714400e+09
3639,33625779332,3.693751e+09
35261,33763859490,3.571675e+09
3742,33626320676,3.429428e+09
25241,33669054076,3.424131e+09
26877,33675877202,3.404278e+09
